<a href="https://colab.research.google.com/github/slala2121/Triplet-net-keras/blob/COS597D/deep_metric_learning_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code adapted from:
https://github.com/KinWaiCheuk/Triplet-net-keras

Other relevant links:

scratch classification network from https://keras.io/examples/cifar10_resnet/

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install --upgrade tensorflow
!pip install tensorflow-addons


     |████████████████████████████████| 86.3MB 36kB/s 
     |████████████████████████████████| 3.8MB 56.3MB/s 
     |████████████████████████████████| 450kB 63.1MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstall

     |████████████████████████████████| 1.8MB 4.8MB/s 
     |████████████████████████████████| 380.8MB 40kB/s 


In [1]:
# current work around for fixing the lifted structure loss file

%%writefile /usr/local/lib/python3.6/dist-packages/tensorflow_addons/losses/lifted.py


# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Implements lifted_struct_loss."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow_addons.losses import metric_learning
from tensorflow_addons.utils import keras_utils


@keras_utils.register_keras_custom_object
@tf.function
def lifted_struct_loss(labels, embeddings, margin=1.0):
    """Computes the lifted structured loss.

    Args:
      labels: 1-D tf.int32 `Tensor` with shape [batch_size] of
        multiclass integer labels.
      embeddings: 2-D float `Tensor` of embedding vectors. Embeddings should
        not be l2 normalized.
      margin: Float, margin term in the loss definition.

    Returns:
      lifted_loss: tf.float32 scalar.
    """
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    lshape = tf.shape(labels)
    # assert lshape.shape == 1
    labels = tf.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pairwise_distances = metric_learning.pairwise_distance(embeddings)

    # Build pairwise binary adjacency matrix.
    adjacency = tf.math.equal(labels, tf.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = tf.math.logical_not(adjacency)

    batch_size = tf.size(labels)

    diff = margin - pairwise_distances
    mask = tf.cast(adjacency_not, dtype=tf.dtypes.float32)
    # Safe maximum: Temporarily shift negative distances
    #   above zero before taking max.
    #     this is to take the max only among negatives.
    row_minimums = tf.math.reduce_min(diff, 1, keepdims=True)
    row_negative_maximums = tf.math.reduce_max(
        tf.math.multiply(diff - row_minimums, mask), 1,
        keepdims=True) + row_minimums

    # Compute the loss.
    # Keep track of matrix of maximums where M_ij = max(m_i, m_j)
    #   where m_i is the max of alpha - negative D_i's.
    # This matches the Caffe loss layer implementation at:
    #   https://github.com/rksltnl/Caffe-Deep-Metric-Learning-CVPR16/blob/0efd7544a9846f58df923c8b992198ba5c355454/src/caffe/layers/lifted_struct_similarity_softmax_layer.cpp  # pylint: disable=line-too-long

    max_elements = tf.math.maximum(row_negative_maximums,
                                   tf.transpose(row_negative_maximums))
    diff_tiled = tf.tile(diff, [batch_size, 1])
    mask_tiled = tf.tile(mask, [batch_size, 1])
    max_elements_vect = tf.reshape(tf.transpose(max_elements), [-1, 1])

    loss_exp_left = tf.reshape(
        tf.math.reduce_sum(
            tf.math.multiply(
                tf.math.exp(diff_tiled - max_elements_vect), mask_tiled),
            1,
            keepdims=True), [batch_size, batch_size])

    loss_mat = max_elements + tf.math.log(loss_exp_left +
                                          tf.transpose(loss_exp_left))
    # Add the positive distance.
    loss_mat += pairwise_distances

    mask_positives = tf.cast(
        adjacency, dtype=tf.dtypes.float32) - tf.linalg.diag(
            tf.ones([batch_size]))

    # *0.5 for upper triangular, and another *0.5 for 1/2 factor for loss^2.
    num_positives = tf.math.reduce_sum(mask_positives) / 2.0

    lifted_loss = tf.math.truediv(
        0.25 * tf.math.reduce_sum(
            tf.math.square(
                tf.math.maximum(
                    tf.math.multiply(loss_mat, mask_positives), 0.0))),
        num_positives)
    return lifted_loss


@keras_utils.register_keras_custom_object
class LiftedStructLoss(tf.keras.losses.Loss):
    """Computes the lifted structured loss.

    The loss encourages the positive distances (between a pair of embeddings
    with the same labels) to be smaller than any negative distances (between
    a pair of embeddings with different labels) in the mini-batch in a way
    that is differentiable with respect to the embedding vectors.
    See: https://arxiv.org/abs/1511.06452.

    Args:
      margin: Float, margin term in the loss definition.
      name: Optional name for the op.
    """

    def __init__(self, margin=1.0, name=None):
        super(LiftedStructLoss, self).__init__(
            name=name, reduction=tf.keras.losses.Reduction.NONE)
        self.margin = margin

    def call(self, y_true, y_pred):
        return lifted_struct_loss(y_true, y_pred, self.margin)

    def get_config(self):
        config = {
            "margin": self.margin,
        }
        base_config = super(LiftedStructLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))



Overwriting /usr/local/lib/python3.6/dist-packages/tensorflow_addons/losses/lifted.py


In [0]:
import os
os.kill(os.getpid(), 9)

In [0]:
import time
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import random
from itertools import permutations
from PIL import Image

import tensorflow as tf
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optimizers

from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint

import tensorflow_addons as tfa


source_path=os.path.join('drive','My Drive', 'Colab Notebooks')


In [3]:
# prepare dataset either for classification or deep metric learning
import tensorflow_datasets as tfds

dataset_name='cifar10'

dataset_dir=os.path.join(source_path,dataset_name)
if not os.path.isdir(dataset_dir):
  os.mkdir(dataset_dir)

debug=1

if debug:
  split_percent=1
  train_split = tfds.Split.TRAIN.subsplit(tfds.percent[:split_percent])
  test_split = tfds.Split.TEST.subsplit(tfds.percent[:split_percent])
  train_dataset,info = tfds.load(name=dataset_name, split=train_split, as_supervised=True, with_info=True)
  test_dataset,info = tfds.load(name=dataset_name, split=test_split, as_supervised=True, with_info=True)
else:
  train_dataset,info = tfds.load(name=dataset_name, split='train', as_supervised=True, with_info=True)
  test_dataset,info = tfds.load(name=dataset_name, split='test', as_supervised=True, with_info=True)

input_dim=info.features['image'].shape
num_classes=info.features['label'].num_classes

train_mean_path=os.path.join(dataset_dir,'train_mean.npy')
if os.path.exists(train_mean_path):
  train_mean=np.load(train_mean_path)
else:
  train_mean=[]
  num_train_images=info.splits['train'].num_examples
  train_mean=[]
  for example in train_dataset.take(num_train_images):
    image,label=example[0],example[1]
    image=image.numpy().astype('float32')
    if len(train_mean)==0:
      train_mean=image
    else:
      train_mean = train_mean+image

  train_mean=train_mean*1.0/num_train_images
  np.save(train_mean_path,train_mean)








/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/1.0.2. Subsequent calls will reuse this data.


In [0]:
# prepare dataset either for classification or deep metric learning

def _normalize_img(img, label):
    img = img - train_mean
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)


# preprocessing of labels for classification

def _encode_one_hot(img, label):
    label = tf.one_hot(label,num_classes)
    return (img, label)


train_dataset = train_dataset.map(_normalize_img)
test_dataset = test_dataset.map(_normalize_img)

loss_type='triplet'
if loss_type=='classification':
  train_dataset = train_dataset.map(_encode_one_hot)
  test_dataset = test_dataset.map(_encode_one_hot)

# Build your input pipelines

train_dataset = train_dataset.shuffle(1024).batch(32)
test_dataset = test_dataset.batch(32)

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# based on the lifted scheme paper
def create_deep_base_network(input_dim,loss_type,num_classes=0,transfer=False,freeze_weights=False):
  weights='imagenet' if transfer else None
  conv_base = ResNet50(weights=weights, include_top=False, input_shape=input_dim)

  if freeze_weights:
    for layer in conv_base.layers:
      layer.trainable=False
  
  model = models.Sequential()
  model.add(layers.Input(input_dim))
  model.add(conv_base)

  model.add(layers.Flatten())
  model.add(layers.BatchNormalization())

  if loss_type=='classification':
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(10, activation='softmax'))
  else:
    model.add(layers.Dense(64, activation=None))
    model.add(layers.Dropout(0.5))

  return model


def create_shallow_network(input_dim,loss_type,num_classes=0):
  model=tf.keras.Sequential()
  model.add(layers.Input(input_dim))
  model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=5, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
  model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=2, padding='same'))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.BatchNormalization())

  if loss_type=='classification':
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))
  else:
    model.add(tf.keras.layers.Dense(256, activation=None))
    model.add(tf.keras.layers.Dropout(0.5))

  return model

def construct_model(model_type,input_dim,loss_type,num_classes,transfer=False,freeze_weights=False):
  if model_type=='shallow':
    model=create_shallow_network(input_dim,loss_type,num_classes)
  elif model_type=='deep':
    model=create_deep_base_network(input_dim,loss_type,num_classes,transfer,freeze_weights)

  if loss_type =='triplet':
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
  return model

def construct_loss(loss_type,margin):
  if loss_type=='triplet':
    loss=tfa.losses.TripletSemiHardLoss(margin=margin)
  elif loss_type=='lifted':
    loss=tfa.losses.LiftedStructLoss(margin=margin)
  elif loss_type=='classification':
    loss=tf.keras.losses.CategoricalCrossentropy()
  return loss

In [9]:
# tune lr

lrs=[1e-1,1e-2,1e-3,1e-4,1e-5]
model_type='deep'
transfer=True
freeze_weights=False

margin=1.0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=30

fig,ax=plt.subplots(2,3)
ax=ax.ravel()
for lr_index,lr in enumerate(lrs):

  model=construct_model(model_type,input_dim,loss_type,num_classes,transfer,freeze_weights)
  loss=construct_loss(loss_type,margin)

  # Compile the model
  model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss)

  history = model.fit(
      train_dataset,
      epochs=num_epochs)

  ax[lr_index].set_title('Loss for lr_%s'%(str(lr)))
  ax[lr_index].plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss_lr_%s'%(str(lr)))
  
plt.legend()
plt.savefig(os.path.join(model_dir,'loss_plot_lr_tune_%s_transfer_%s_freeze_%s_%s_%s.png'%(model_type,transfer,freeze_weights,loss_type,margin)))
plt.close()

Epoch 1/30
16/16 [==============================] - 13s 809ms/step - loss: 0.9656
Epoch 2/30
16/16 [==============================] - 3s 157ms/step - loss: 0.9507
Epoch 3/30
16/16 [==============================] - 2s 149ms/step - loss: 0.9472
Epoch 4/30
16/16 [==============================] - 2s 155ms/step - loss: 0.9236
Epoch 5/30
16/16 [==============================] - 3s 156ms/step - loss: 0.9253
Epoch 6/30
16/16 [==============================] - 2s 153ms/step - loss: 0.9242
Epoch 7/30
16/16 [==============================] - 3s 157ms/step - loss: 0.9211
Epoch 8/30
16/16 [==============================] - 2s 156ms/step - loss: 0.9143
Epoch 9/30
16/16 [==============================] - 3s 159ms/step - loss: 0.9148
Epoch 10/30
16/16 [==============================] - 3s 159ms/step - loss: 0.8932
Epoch 11/30
16/16 [==============================] - 2s 155ms/step - loss: 0.9188
Epoch 12/30
16/16 [==============================] - 2s 156ms/step - loss: 0.8995
Epoch 13/30
16/16 [=====

In [14]:
# train model

lr=1e-3

model_type='deep'
transfer=True
freeze_weights=False

margin=1.0

model_dir=os.path.join(dataset_dir,model_type)
if not os.path.isdir(model_dir):
  os.mkdir(model_dir)

num_epochs=30

model=construct_model(model_type,input_dim,loss_type,num_classes,transfer,freeze_weights)
loss=construct_loss(loss_type,margin)

# Compile the model
metrics=[]
if loss_type=='classification':
  metrics=['accuracy']
model.compile(optimizer=tf.keras.optimizers.Adam(lr),loss=loss,metrics=metrics)

# Prepare callbacks for model saving and for learning rate adjustment.
filepath=os.path.join(model_dir,'final_%s_%s_transfer_%s_freeze_%s_margin_%s.h5'%(loss_type,model_type,transfer,freeze_weights,margin))
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint]

history = model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=test_dataset,
    callbacks=callbacks)


plt.figure()
plt.plot(np.arange(num_epochs),history.history['loss'],'r',label='train_loss')
plt.plot(np.arange(num_epochs),history.history['val_loss'],'b',label='val_loss')
plt.legend()
filepath=os.path.join(model_dir,'final_loss_plot_%s_%s_transfer_%s_freeze_%s_margin_%s.png'%(loss_type,model_type,transfer,freeze_weights,margin))
plt.savefig(filepath)
plt.close()

Epoch 1/30
   1563/Unknown - 85s 54ms/step - loss: 0.9060
Epoch 00001: val_loss improved from inf to 0.90591, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_triplet_deep_transfer_True_freeze_False_margin_1.0.h5
1563/1563 [==============================] - 93s 59ms/step - loss: 0.9060 - val_loss: 0.0000e+00
Epoch 2/30
312/313 [============================>.] - ETA: 0s - loss: 0.8964
Epoch 00002: val_loss improved from 0.90591 to 0.85410, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_triplet_deep_transfer_True_freeze_False_margin_1.0.h5
1563/1563 [==============================] - 79s 51ms/step - loss: 0.8899 - val_loss: 0.8541
Epoch 3/30
311/313 [============================>.] - ETA: 0s - loss: 0.8807
Epoch 00003: val_loss improved from 0.85410 to 0.84116, saving model to drive/My Drive/Colab Notebooks/cifar10/deep/final_triplet_deep_transfer_True_freeze_False_margin_1.0.h5
1563/1563 [==============================] - 79s 51ms/step - loss: 0.8737 